In [5]:
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np

# loadin Data

In [2]:
mo
from importlib import reload

In [3]:
reload(data)
Dataset = data.DataBinary

In [4]:
ds = Dataset('./Data/4-15-divided-tokenized-vectored/test/')

loading from: ./Data/4-15-divided-tokenized-vectored/test/
loaded


In [5]:
step = 0
for x,y in ds.get_batch(batch_size=3):
#     print(y)
    print(x[0].shape)
    step += 1
    if step > 10:
        break

(3, 14, 1024)
(3, 21, 1024)
(3, 33, 1024)
(3, 12, 1024)
(3, 49, 1024)
(3, 9, 1024)
(3, 50, 1024)
(3, 29, 1024)
(3, 18, 1024)
(3, 36, 1024)
(3, 23, 1024)


In [6]:
print(len(ds.data))

69965


# loading model

In [7]:
from code import model

In [295]:
reload(model)
RNN = model.RNN
rnn = RNN(1024,3,nb_layers=2,nb_units=1024)
rnn.cuda()

RNN(
  (rnn): GRU(1024, 1024, num_layers=2, batch_first=True)
  (linear1): Linear(in_features=1024, out_features=1024, bias=True)
  (linear2): Linear(in_features=1024, out_features=3, bias=True)
  (relu): ReLU()
)

# auxiliary

In [267]:
def pad_data(s, maxlen):
    padded = np.zeros((maxlen,s.shape[1]), dtype=np.double)
    if len(s) > maxlen: padded[:] = s[:maxlen]
    else: padded[:len(s)] = s
    return padded

def prepare_batch(x,y):
    x.sort(key=lambda item: -item.shape[1])
    x_len = []
    for i in range(len(x)):
        l = min(x[i].shape[1],50)
        x[i] = pad_data(x[i][0],50)
        x_len.append(l)
    x = np.array(x)
    x_len = np.array(x_len)
    x = torch.from_numpy(x).float().cuda()
    x_len = torch.from_numpy(x_len).float()
    y = torch.from_numpy(np.array(y,dtype=int)).cuda()
    return x, x_len, y


In [294]:
def train_epoch(model, batch_gen, batch_size, ds, epoch = 1):
    loss_fn = nn.CrossEntropyLoss()
    total_epoch_loss = 0
    total_epoch_acc = 0
    model.cuda()
#     optim = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),lr = 0.001)
    optim = torch.optim.Adam(model.parameters(), lr=0.1)
#     optim = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    steps = 0
    model.train()
    for x, y in batch_gen(ds, batch_size=batch_size):
        x,x_len,y = prepare_batch(x,y)
        y = torch.autograd.Variable(y).long()
#         optim.zero_grad()
        model.zero_grad()
        y_hat = model(x,x_len)
        loss = loss_fn(y_hat,y)
        num_currect = (torch.max(y_hat, 1)[1].view(y.size()).data == y.data).float().sum()
        acc = 100.0 * num_currect/batch_size
        loss.backward()
        optim.step()
        
        steps += 1
        total_epoch_loss += loss.item()
        total_epoch_acc += acc.item()
        
        if steps % 100 == 0:
            print (f'Epoch: {epoch+1}, batch: {steps}, Training Loss: {total_epoch_loss/steps:.4f}, Training Accuracy: {total_epoch_acc/steps: .2f}%')
            
        
    return total_epoch_loss/steps, total_epoch_acc/steps

In [251]:
def get_batch(ds, batch_size=32, shuffeling=True):
    indices = np.arange(len(ds.data))
    d = []
    s = 0
    for i in indices:
        if ds.labels[i] == '1':
            s +=1
            if np.random.rand() > 0.2:
                d.append(i)
    indices = np.delete(indices,d)
#     print(len(ds.data),len(indices),len(d),s)
    if shuffeling:
        np.random.shuffle(indices)
    for i in range(0,len(indices),batch_size):
        d = indices[i:i+batch_size]
        x = [ds.data[i] for i in d]
        y = [ds.labels[i] for i in d]
        yield x,y

In [296]:

for i in range(5):
    print(train_epoch(rnn,get_batch,64,ds,i))

Epoch: 1, batch: 100, Training Loss: 2.9719, Training Accuracy:  33.50%
Epoch: 1, batch: 200, Training Loss: 4.6278, Training Accuracy:  33.02%
Epoch: 1, batch: 300, Training Loss: 3.4595, Training Accuracy:  33.36%
Epoch: 1, batch: 400, Training Loss: 2.8708, Training Accuracy:  33.37%
(2.600925504403599, 33.23622881355932)
Epoch: 2, batch: 100, Training Loss: 1.1436, Training Accuracy:  33.45%
Epoch: 2, batch: 200, Training Loss: 1.1356, Training Accuracy:  33.57%
Epoch: 2, batch: 300, Training Loss: 1.1246, Training Accuracy:  33.97%
Epoch: 2, batch: 400, Training Loss: 1.1190, Training Accuracy:  33.73%
(1.1165214696001349, 33.718152866242036)


KeyboardInterrupt: 

In [299]:
i = 0
for x, y in ds.get_batch(batch_size=10):
#         print(len(x))
    x,x_len,y = prepare_batch(x,y)
#         print(steps,x.shape)
#     optim.zero_grad()
    y_hat = rnn(x,x_len)
    i += 1
    if i > 0:
        break

In [300]:
print(y_hat.data)
print(y_hat.cpu().data.numpy().argmax(axis=1))
print(y)

tensor([[ 0.0433,  0.1417, -0.0017],
        [ 0.0433,  0.1417, -0.0017],
        [ 0.0433,  0.1417, -0.0017],
        [ 0.0433,  0.1417, -0.0017],
        [ 0.0433,  0.1417, -0.0017],
        [ 0.0433,  0.1417, -0.0017],
        [ 0.0433,  0.1417, -0.0017],
        [ 0.0433,  0.1417, -0.0017],
        [ 0.0433,  0.1417, -0.0017],
        [ 0.0433,  0.1417, -0.0017]], device='cuda:0')
[1 1 1 1 1 1 1 1 1 1]
tensor([2, 1, 2, 1, 1, 1, 1, 0, 2, 1], device='cuda:0')


In [91]:
s = {0:0,1:0,2:0}
for i in ds.labels:
    s[int(i)] += 1
print(s)

{0: 10000, 1: 49965, 2: 10000}


In [25]:
print(F.cross_entropy(y_hat, y))

tensor(0.9514, device='cuda:0', grad_fn=<NllLossBackward>)


In [28]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
print(output)

tensor(1.8375, grad_fn=<NllLossBackward>)


In [30]:
print(input)
print(target)

tensor([[ 0.4739, -0.8596, -0.8077, -0.3942,  0.4430],
        [ 1.1719,  0.5002, -0.5580, -0.3177,  0.5473],
        [-0.2881, -1.8407,  0.9588, -0.6890, -1.6679]], requires_grad=True)
tensor([1, 2, 2])


In [31]:
a = np.array([1,2,3,4,5,6,7,8])
print(a)
a = np.delete(a,[1,2])
print(a)

[1 2 3 4 5 6 7 8]
[1 4 5 6 7 8]
